In [466]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform
import numpy as np
import os
import matplotlib.pyplot as plt
from skimage.transform import resize
from glob import glob
from tqdm import tqdm
from torchmetrics.classification import BinaryAUROC
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [467]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [468]:
def RetinaTransform(image):
    # Bringing image to 0-1 range, #ATTENTION: CHECK IF ALL IMAGES min max is 0 and 255
    image = image / 255
    
    
    image = resize(image, (64,64),anti_aliasing=False) # resizing image because original does not fit in memory.
    
    
    # Bringing data to CHW format
    # N is a batch size, C denotes a number of channels, 
    # H is a height of input planes in pixels, and W is width in pixels.
    image = image.transpose([2,0,1])
    
    #Fixing dtype to avoid runtime error and save memory
    image = torch.tensor(image ,dtype=torch.float32)
    
    return image

In [469]:
# # Load dataset into memory
# cuda_image_array = []
# data_folder = "Evaluation_Set/Validation"
# label_path = "Evaluation_Set/RFMiD_Validation_Labels.csv"
# label_frame = pd.read_csv(label_path)
# local_transform = RetinaTransform

# for image_name in tqdm(glob(data_folder+"/*")):
#     image = io.imread(image_name)

#     label_frame_index = int(image_name.split("/")[-1].split(".")[0])
#     label = label_frame[label_frame["ID"]==label_frame_index]["Disease_Risk"].values[0]

#     image = local_transform(image)
#     label =  torch.tensor(label ,dtype=torch.float32)

#     cuda_image_array.append((image.to(device),label.to(device)))

In [519]:
class CicikNet(nn.Module):
    def __init__(self):
        super(CicikNet, self).__init__()

        
        self.lrelu = nn.LeakyReLU(0.1)
        # First 2D convolutional layer, taking in 1 input channel (image),
        # outputting 32 convolutional features, with a square kernel size of 3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1,padding=1)
        # Second 2D convolutional layer, taking in the 32 input layers,
        # outputting 64 convolutional features, with a square kernel size of 3
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=1, kernel_size=5, stride=1,padding=2)


        # First fully connected layer
        self.fc1 = nn.Linear(64*64+64*64, 128)
        # Second fully connected layer that outputs our 10 labels
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x1 = torch.flatten(self.conv1(x),1)
        x2 = torch.flatten(self.conv2(x),1)
        
        x = torch.cat([x1,x2],dim=1)

        x = self.fc1(x)
        x = self.lrelu(x)

        
        x = self.fc2(x)
        
        output = torch.sigmoid(x)
        return output

In [520]:
class RetinaDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, data_indices, transform=None):
        
        self.data = [cuda_image_array[idx] for idx in data_indices]
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [522]:
# def plot_grad_flow(named_parameters):
#     ave_grads = []
#     layers = []
#     for n, p in named_parameters:
#         if(p.requires_grad) and ("bias" not in n):
#             layers.append(n)
#             ave_grads.append(p.grad.abs().mean())
#     plt.plot(ave_grads, alpha=0.3, color="b")
#     plt.hlines(0, 0, len(ave_grads)+1, linewidth=1, color="k" )
#     plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
#     plt.xlim(xmin=0, xmax=len(ave_grads))
#     plt.xlabel("Layers")
#     plt.ylabel("average gradient")
#     plt.title("Gradient flow")
#     plt.grid(True)

In [523]:
def model_train_loop(net,train_dataloader):
    
    running_loss = 0.0
    predictions = []
    ground_truth = []
    
    for idx, data in enumerate(train_dataloader, 0):
        # get the inputs
        # print(i,data)
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs.flatten(), labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm=10)
        
        optimizer.step()
        running_loss += loss.item()
        
        
        
        predictions.append(outputs.flatten())
        ground_truth.append(labels.flatten())
        
    # net.to('cpu')
    # plot_grad_flow(net.named_parameters())
    # plt.show()
    # net.to(device)
    
    predictions=torch.cat(predictions)
    ground_truth = torch.cat(ground_truth)
    
    
    healthy_f1 = f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)
    rocauc = bauc(predictions,ground_truth)
    print('[%d, %5d] loss: %.3f, ROC: %.3f, F1_healthy: %.3f\n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))

In [524]:
def model_test_loop(net, test_dataloader):
    
    with torch.no_grad():
        
        running_loss = 0.0
        test_predictions = []
        test_ground_truth = []
        
        for idx, data in enumerate(test_dataloader, 0):

            inputs, labels = data

            outputs = net(inputs)
            loss = criterion(outputs.flatten(), labels)
            running_loss += loss.item()

            test_predictions.append(outputs.flatten())
            test_ground_truth.append(labels.flatten())

        
        
        test_predictions=torch.cat(test_predictions)
        test_ground_truth = torch.cat(test_ground_truth)


        healthy_f1 = f1_score((test_predictions>0.5).float().cpu(),test_ground_truth.cpu(),pos_label=0)
        rocauc = bauc(test_predictions,test_ground_truth)
        # print('[%d, %5d] Validation loss: %.3f, ROC: %.3f, F1_healthy: %.3f \n' % (epoch + 1, idx + 1, running_loss, rocauc, healthy_f1))
        
        return rocauc,healthy_f1


In [525]:
# f1_score((predictions>0.5).float().cpu(),ground_truth.cpu(),pos_label=0)

In [526]:
# print(len(train_dataloader))

In [527]:
# MODEL learns the trivial case of all 1s, which means it has capacity to learn:


    
# print('Finished Training')

In [539]:
# EPOCHS = 100
# criterion = nn.BCELoss()
# bauc = BinaryAUROC(thresholds=None)

# kf = KFold(n_splits=5)

# fold_rocauc = []
# fold_f1healthy = []

# for idx, (train_index, test_index) in tqdm(enumerate(kf.split(cuda_image_array))):
#     train_dataset = RetinaDataset(data_indices=train_index)
#     test_dataset = RetinaDataset(data_indices=test_index)
    
#     train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=False, num_workers=0)
#     test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=0)
    
#     ciciknet = CicikNet().to(device)
#     optimizer = optim.SGD(ciciknet.parameters(), lr=0.001, momentum=0.9)
    
#     for epoch in range(EPOCHS):  # loop over the dataset multiple times
#         model_train_loop(ciciknet,train_dataloader)
    
#     rocauc, f1healthy = model_test_loop(ciciknet,test_dataloader)
    
#     fold_rocauc.append(rocauc)
#     fold_f1healthy.append(f1healthy)

0it [00:00, ?it/s]

[1,    52] loss: 29.956, ROC: 0.435, F1_healthy: 0.000

[2,    52] loss: 26.324, ROC: 0.570, F1_healthy: 0.000

[3,    52] loss: 26.313, ROC: 0.569, F1_healthy: 0.000

[4,    52] loss: 26.087, ROC: 0.582, F1_healthy: 0.000

[5,    52] loss: 25.864, ROC: 0.595, F1_healthy: 0.000

[6,    52] loss: 25.669, ROC: 0.610, F1_healthy: 0.000

[7,    52] loss: 25.501, ROC: 0.621, F1_healthy: 0.000

[8,    52] loss: 25.333, ROC: 0.630, F1_healthy: 0.000

[9,    52] loss: 25.180, ROC: 0.640, F1_healthy: 0.000

[10,    52] loss: 25.020, ROC: 0.647, F1_healthy: 0.000

[11,    52] loss: 24.860, ROC: 0.654, F1_healthy: 0.000

[12,    52] loss: 24.700, ROC: 0.661, F1_healthy: 0.000

[13,    52] loss: 24.545, ROC: 0.668, F1_healthy: 0.000

[14,    52] loss: 24.384, ROC: 0.674, F1_healthy: 0.000

[15,    52] loss: 24.226, ROC: 0.683, F1_healthy: 0.000

[16,    52] loss: 24.062, ROC: 0.691, F1_healthy: 0.019

[17,    52] loss: 23.908, ROC: 0.701, F1_healthy: 0.019

[18,    52] loss: 23.756, ROC: 0.709, F1

1it [00:09,  9.20s/it]

[99,    52] loss: 9.012, ROC: 0.967, F1_healthy: 0.851

[100,    52] loss: 8.865, ROC: 0.968, F1_healthy: 0.860

[1,    52] loss: 29.102, ROC: 0.510, F1_healthy: 0.000

[2,    52] loss: 27.602, ROC: 0.563, F1_healthy: 0.000

[3,    52] loss: 27.381, ROC: 0.585, F1_healthy: 0.000

[4,    52] loss: 27.203, ROC: 0.604, F1_healthy: 0.000

[5,    52] loss: 27.033, ROC: 0.619, F1_healthy: 0.000

[6,    52] loss: 26.867, ROC: 0.634, F1_healthy: 0.000

[7,    52] loss: 26.699, ROC: 0.646, F1_healthy: 0.000

[8,    52] loss: 26.528, ROC: 0.657, F1_healthy: 0.000

[9,    52] loss: 26.355, ROC: 0.668, F1_healthy: 0.000

[10,    52] loss: 26.174, ROC: 0.678, F1_healthy: 0.000

[11,    52] loss: 25.977, ROC: 0.686, F1_healthy: 0.000

[12,    52] loss: 25.778, ROC: 0.695, F1_healthy: 0.000

[13,    52] loss: 25.590, ROC: 0.703, F1_healthy: 0.000

[14,    52] loss: 25.410, ROC: 0.710, F1_healthy: 0.017

[15,    52] loss: 25.237, ROC: 0.716, F1_healthy: 0.033

[16,    52] loss: 25.044, ROC: 0.723, F1_

2it [00:18,  9.32s/it]

...

[100,    52] loss: 9.409, ROC: 0.970, F1_healthy: 0.833

[1,    52] loss: 27.796, ROC: 0.543, F1_healthy: 0.018

[2,    52] loss: 26.445, ROC: 0.566, F1_healthy: 0.000

[3,    52] loss: 26.329, ROC: 0.572, F1_healthy: 0.000

[4,    52] loss: 26.244, ROC: 0.578, F1_healthy: 0.000

[5,    52] loss: 26.158, ROC: 0.584, F1_healthy: 0.000

[6,    52] loss: 26.068, ROC: 0.592, F1_healthy: 0.000

[7,    52] loss: 25.968, ROC: 0.600, F1_healthy: 0.000

[8,    52] loss: 25.870, ROC: 0.605, F1_healthy: 0.000

[9,    52] loss: 25.764, ROC: 0.612, F1_healthy: 0.000

[10,    52] loss: 25.632, ROC: 0.622, F1_healthy: 0.000

[11,    52] loss: 25.508, ROC: 0.630, F1_healthy: 0.000

[12,    52] loss: 25.359, ROC: 0.639, F1_healthy: 0.000

[13,    52] loss: 25.211, ROC: 0.646, F1_healthy: 0.000

[14,    52] loss: 25.041, ROC: 0.653, F1_healthy: 0.000

[15,    52] loss: 24.898, ROC: 0.658, F1_healthy: 0.000

[16,    52] loss: 24.736, ROC: 0.665, F1_healthy: 0.000

[17,    52] loss: 24.586, ROC: 0.672, F1

4it [00:36,  9.08s/it]

[98,    52] loss: 9.525, ROC: 0.967, F1_healthy: 0.792

[99,    52] loss: 9.303, ROC: 0.969, F1_healthy: 0.814

[100,    52] loss: 9.104, ROC: 0.971, F1_healthy: 0.828

[1,    52] loss: 30.555, ROC: 0.475, F1_healthy: 0.000

[2,    52] loss: 26.024, ROC: 0.554, F1_healthy: 0.000

[3,    52] loss: 26.051, ROC: 0.561, F1_healthy: 0.000

[4,    52] loss: 25.916, ROC: 0.573, F1_healthy: 0.000

[5,    52] loss: 25.767, ROC: 0.586, F1_healthy: 0.000

[6,    52] loss: 25.615, ROC: 0.598, F1_healthy: 0.000

[7,    52] loss: 25.473, ROC: 0.609, F1_healthy: 0.000

[8,    52] loss: 25.322, ROC: 0.620, F1_healthy: 0.000

[9,    52] loss: 25.163, ROC: 0.632, F1_healthy: 0.000

[10,    52] loss: 24.998, ROC: 0.642, F1_healthy: 0.000

[11,    52] loss: 24.817, ROC: 0.653, F1_healthy: 0.000

[12,    52] loss: 24.624, ROC: 0.664, F1_healthy: 0.000

[13,    52] loss: 24.413, ROC: 0.673, F1_healthy: 0.000

[14,    52] loss: 24.193, ROC: 0.683, F1_healthy: 0.000

[15,    52] loss: 23.960, ROC: 0.693, F1_h

5it [00:45,  9.16s/it]

[99,    52] loss: 7.076, ROC: 0.984, F1_healthy: 0.865

[100,    52] loss: 6.908, ROC: 0.985, F1_healthy: 0.859



In [540]:
print(fold_rocauc)

[tensor(0.8736, device='cuda:0'), tensor(0.8384, device='cuda:0'), tensor(0.7858, device='cuda:0'), tensor(0.8760, device='cuda:0'), tensor(0.8380, device='cuda:0')]


In [541]:
print(fold_f1healthy)

[0.6885245901639345, 0.5, 0.4583333333333333, 0.5818181818181819, 0.5964912280701754]
